<a href="https://colab.research.google.com/github/S-Abhishekk/Celebal-Assignments-/blob/main/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using the breast cancer classification with hyperparameter tuning
In this notebook:
- Trained Logistic Regression, Random Forest, and SVM machine learning models
- Evaluated them using Accuracy, Precision, Recall, F1 Score
- Tuned parameters using GridSearchCV and RandomizedSearchCV
- In the end selected the best-performing model

In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [2]:
#Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [3]:
#Model Training & Evaluation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

models = {
    'Logistic Regression':LogisticRegression(),
    'Random Forest':RandomForestClassifier(),
    'Support Vector Machine':SVC()
}
def evaluate_model(name, model):
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    return {
        'Model': name,
        'Accuracy':accuracy_score(y_test, y_pred),
        'Precision':precision_score(y_test, y_pred),
        'Recall':recall_score(y_test, y_pred),
        'F1 Score':f1_score(y_test, y_pred)
    }
results = [evaluate_model(name, model) for name, model in models.items()]
df_results = pd.DataFrame(results)
df_results

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.973684,0.972222,0.985915,0.979021
1,Random Forest,0.964912,0.958904,0.985915,0.972222
2,Support Vector Machine,0.982456,0.972603,1.000000,0.986111


In [4]:
#Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
param_grid_rf = {
    'n_estimators':[50, 100],
    'max_depth':[None, 5, 10],
    'min_samples_split':[2, 5]
}
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=3, scoring='f1')
grid_search_rf.fit(X_train_scaled, y_train)
param_dist_svc = {
    'C':[0.1, 5, 50],
    'gamma':['scale', 0.001, 0.02, 0.3],
    'kernel':['linear', 'rbf']
}
random_search_svc = RandomizedSearchCV(SVC(), param_distributions=param_dist_svc, n_iter=5, cv=3, scoring='f1')
random_search_svc.fit(X_train_scaled, y_train)

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=5,
                   param_distributions={'C': [0.1, 5, 50],
                                        'gamma': ['scale', 0.001, 0.02, 0.3],
                                        'kernel': ['linear', 'rbf']},
                   scoring='f1')

In [5]:
#Evaluate Best Tuned Models
best_rf = grid_search_rf.best_estimator_
best_svc = random_search_svc.best_estimator_
tuned_results = [
    evaluate_model("Tuned Random Forest", best_rf),
    evaluate_model("Tuned SVC", best_svc)
]
df_tuned_results = pd.DataFrame(tuned_results)
final_results = pd.concat([df_results, df_tuned_results], ignore_index=True)
final_results.sort_values(by="F1 Score", ascending=False)

,Model,Accuracy,Precision,Recall,F1 Score
2,Support Vector Machine,0.982456,0.972603,1.000000,0.986111
4,Tuned SVC,0.982456,0.972603,1.000000,0.986111
0,Logistic Regression,0.973684,0.972222,0.985915,0.979021
1,Random Forest,0.964912,0.958904,0.985915,0.972222
3,Tuned Random Forest,0.964912,0.958904,0.985915,0.972222


-Both the untuned and tuned SVM models achieved the highest F1-score of 0.9861 and perfect recall of 1.0, means they identified all malignant cases correctly.
-This makes SVM an ideal choice for medical diagnostics where minimizing false negatives is crucial.
-Hyperparameter tuning improved Random Forest a little but did not significantly change SVM’s performance, indicating the base SVM was already near-optimal for this dataset.
-In contrast, the tuned Random Forest underperformed compared to its untuned version, likely due to overfitting or inappropriate parameter ranges.